# Back Test Analysis

This notebook implements a comprehensive pairs trading strategy for BTC-USDT and ETH-USDT cryptocurrency pairs. It incorporates robust statistical analysis, dynamic hedge ratio calculation, and complete backtesting framework.

## 1. Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import coint, adfuller
import statsmodels.api as sm
import os

# Set style for better visualizations
plt.style.use('seaborn-v0_8-whitegrid')

# Create docs directory for saving plots
os.makedirs('docs', exist_ok=True)

## 2. Helper Functions

In [ ]:
# Function to run linear regression
def lin_reg(X, Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    params = model.params
    return params

## 3. Load and Prepare Data

In [ ]:
print("Loading BTC-USDT and ETH-USDT data...")

# Load the data
btcusdt = pd.read_csv('../data/btcusdt_1m_raw.csv')
ethusdt = pd.read_csv('../data/ethusdt_1m_raw.csv')

# Basic data exploration
print("BTCUSDT data shape:", btcusdt.shape)
print("ETHUSDT data shape:", ethusdt.shape)

In [ ]:
# Convert timestamp to datetime if not already done
btcusdt['datetime'] = pd.to_datetime(btcusdt['datetime'])
ethusdt['datetime'] = pd.to_datetime(ethusdt['datetime'])

# Set datetime as index
btcusdt.set_index('datetime', inplace=True)
ethusdt.set_index('datetime', inplace=True)

# Check data info
print("BTCUSDT date range:", btcusdt.index.min(), "to", btcusdt.index.max())
print("ETHUSDT date range:", ethusdt.index.min(), "to", ethusdt.index.max())

In [ ]:
# Resample to hourly data to reduce noise
btcusdt_hourly = btcusdt['close'].resample('1H').last()
ethusdt_hourly = ethusdt['close'].resample('1H').last()

# Ensure we're working with common timestamps
common_index = btcusdt_hourly.index.intersection(ethusdt_hourly.index)
btc_aligned = btcusdt_hourly.loc[common_index]
eth_aligned = ethusdt_hourly.loc[common_index]

print(f"Length of aligned hourly data: {len(btc_aligned)}")

## 4. Train-Test Split

We'll split our data into training (80%) and testing (20%) periods to develop our model on historical data and evaluate it on out-of-sample data.

In [ ]:
# Split data into training and testing periods (80% train, 20% test)
train_size = int(len(btc_aligned) * 0.8)
btc_train = btc_aligned[:train_size]
eth_train = eth_aligned[:train_size]
btc_test = btc_aligned[train_size:]
eth_test = eth_aligned[train_size:]

print(f"Train period: {btc_train.index.min()} to {btc_train.index.max()}")
print(f"Test period: {btc_test.index.min()} to {btc_test.index.max()}")

In [ ]:
# Calculate basic statistics for training data
btc_stats = btc_train.describe()
eth_stats = eth_train.describe()

stats_df = pd.DataFrame({
    'BTC-USDT': btc_stats,
    'ETH-USDT': eth_stats
})

print("\nTraining Data Statistics:")
print(stats_df)

## 5. Pair Selection Analysis

In [ ]:
# Normalize price series for visualization
btc_norm = btc_train / btc_train.iloc[0]
eth_norm = eth_train / eth_train.iloc[0]

# Visualize normalized price movements
plt.figure(figsize=(14, 7))
plt.plot(btc_norm.index, btc_norm, label='BTC-USDT (Normalized)')
plt.plot(eth_norm.index, eth_norm, label='ETH-USDT (Normalized)')
plt.title('Normalized Price Movements (Training Period)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Normalized Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/normalized_prices.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate Sum of Squared Differences (SSD)
ssd = ((btc_norm - eth_norm) ** 2).sum()
print(f"\nSum of Squared Differences (SSD): {ssd:.4f}")

# Calculate price correlation
price_correlation = btc_train.corr(eth_train)
print(f"Price correlation between BTC-USDT and ETH-USDT: {price_correlation:.4f}")

In [ ]:
# Test for cointegration
score, pvalue, _ = coint(btc_train, eth_train)
print(f"Cointegration p-value: {pvalue:.4f}")
print(f"Pairs are {'cointegrated' if pvalue < 0.05 else 'not cointegrated'} at 5% significance level")

## 6. Spread Modeling 

Calculate the hedge ratio and analyze the spread between BTC-USDT and ETH-USDT.

In [ ]:
# Perform linear regression to find hedge ratio
intercept, hedge_ratio = lin_reg(btc_train, eth_train)
print(f"\nHedge Ratio: {hedge_ratio:.4f}")
print(f"Intercept: {intercept:.4f}")

# Visualize the relationship with regression line
plt.figure(figsize=(10, 6))
plt.scatter(btc_train, eth_train, alpha=0.5, label='Train Data')
x_line = np.linspace(min(btc_train), max(btc_train), 100)
y_line = hedge_ratio * x_line + intercept
plt.plot(x_line, y_line, 'r', label=f'y = {hedge_ratio:.4f}x + {intercept:.4f}')
plt.title('Linear Regression of ETH-USDT vs BTC-USDT (Train Period)')
plt.xlabel('BTC-USDT Price')
plt.ylabel('ETH-USDT Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/regression.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate spread during training period
train_spread = eth_train - hedge_ratio * btc_train

# Test spread for stationarity
adf_result = adfuller(train_spread.dropna())
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")
print(f"The spread is {'stationary' if adf_result[1] < 0.05 else 'non-stationary'} at 5% significance level")

In [ ]:
# Visualize the training spread
plt.figure(figsize=(12, 6))
plt.plot(train_spread.index, train_spread)
plt.axhline(y=train_spread.mean(), color='r', linestyle='-', label=f'Mean: {train_spread.mean():.2f}')
plt.axhline(y=train_spread.mean() + 2*train_spread.std(), color='g', linestyle='--', label='Upper Threshold (2σ)')
plt.axhline(y=train_spread.mean() - 2*train_spread.std(), color='g', linestyle='--', label='Lower Threshold (2σ)')
plt.title('Spread Between ETH-USDT and BTC-USDT (Train Period)')
plt.xlabel('Date')
plt.ylabel('Spread')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/train_spread.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate z-score with rolling window
train_window = 24  # 24 hours = 1 day rolling window
train_rolling_mean = train_spread.rolling(window=train_window).mean()
train_rolling_std = train_spread.rolling(window=train_window).std()
train_z_score = (train_spread - train_rolling_mean) / train_rolling_std

# Visualize z-score for training period
plt.figure(figsize=(12, 6))
plt.plot(train_z_score.dropna())
plt.axhline(y=0, color='black', linestyle='-')
plt.axhline(y=1.5, color='r', linestyle='--', label='Entry Threshold (1.5)')
plt.axhline(y=-1.5, color='r', linestyle='--')
plt.axhline(y=0.5, color='g', linestyle='--', label='Exit Threshold (0.5)')
plt.axhline(y=-0.5, color='g', linestyle='--')
plt.title('Z-score of BTC-USDT and ETH-USDT Spread (Train Period)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/train_zscore.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Rolling Hedge Ratio and Test Period Analysis

Implement a dynamic hedge ratio calculation for the test period.

In [ ]:
print("\nCalculating rolling hedge ratios for test period...")
window_size = 24  # 24 hours rolling window
rolling_hedge_ratios = pd.Series(index=btc_test.index, dtype=float)
test_spread_mean = pd.Series(index=btc_test.index, dtype=float)
test_spread_std = pd.Series(index=btc_test.index, dtype=float)

for i in range(window_size, len(btc_test) + 1):
    window_btc = btc_test.iloc[i - window_size:i]
    window_eth = eth_test.iloc[i - window_size:i]
    
    # Compute rolling hedge ratio
    _, rolling_hedge = lin_reg(window_btc, window_eth)
    rolling_hedge_ratios.iloc[i - 1] = rolling_hedge
    
    # Compute window spread statistics
    window_spread = window_eth - rolling_hedge * window_btc
    test_spread_mean.iloc[i - 1] = window_spread.mean()
    test_spread_std.iloc[i - 1] = window_spread.std()

In [ ]:
# Calculate spread for test period using rolling hedge ratios
test_spread = eth_test - rolling_hedge_ratios * btc_test

# Visualize test spread and rolling hedge ratio
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot test spread
axes[0].plot(test_spread.index, test_spread, label='Test Spread')
axes[0].plot(test_spread_mean.index, test_spread_mean, label='Rolling Mean', color='red')
axes[0].set_title('Test Spread and Rolling Mean')
axes[0].set_ylabel('Spread')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot rolling hedge ratio
axes[1].plot(rolling_hedge_ratios.index, rolling_hedge_ratios)
axes[1].set_title('Rolling Hedge Ratio')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Hedge Ratio')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('docs/test_spread_and_hedge_ratio.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate z-score (normalized spread)
z_score = (test_spread - test_spread_mean) / test_spread_std

# Visualize normalized spread (z-score)
plt.figure(figsize=(12, 6))
plt.plot(z_score.index, z_score, label='Z-Score')
plt.axhline(y=0, color='black', linestyle='-')
plt.axhline(y=1.5, color='r', linestyle='--', label='Entry Threshold (1.5)')
plt.axhline(y=-1.5, color='g', linestyle='--', label='Entry Threshold (-1.5)')
plt.axhline(y=0.5, color='orange', linestyle='--', label='Exit Threshold (0.5)')
plt.axhline(y=-0.5, color='orange', linestyle='--', label='Exit Threshold (-0.5)')
plt.title('Z-Score of ETH-USDT and BTC-USDT Spread (Test Period)')
plt.xlabel('Date')
plt.ylabel('Z-Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/test_zscore.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Backtesting Framework

Implement a complete backtesting framework with trading rules based on z-score signals.

In [ ]:
print("\nImplementing backtesting framework...")

# Create strategy dataframe
strategy = pd.DataFrame(index=z_score.index)
strategy['BTC'] = btc_test
strategy['ETH'] = eth_test
strategy['Z_Score'] = z_score
strategy['Hedge_Ratio'] = rolling_hedge_ratios
strategy['Signal'] = 0
strategy['Position'] = 0

# Drop any NaN values
strategy = strategy.dropna()

In [ ]:
# Implement trading rules
current_position = 0

for i in range(1, len(strategy)):
    # Enter long position (buy spread) when z-score < -1.5
    if strategy['Z_Score'].iloc[i] < -1.5 and current_position == 0:
        strategy['Signal'].iloc[i] = 1
        current_position = 1
    
    # Enter short position (sell spread) when z-score > 1.5
    elif strategy['Z_Score'].iloc[i] > 1.5 and current_position == 0:
        strategy['Signal'].iloc[i] = -1
        current_position = -1
    
    # Exit long position when z-score crosses above 0.5
    elif strategy['Z_Score'].iloc[i] > 0.5 and current_position == 1:
        strategy['Signal'].iloc[i] = -1
        current_position = 0
    
    # Exit short position when z-score crosses below -0.5
    elif strategy['Z_Score'].iloc[i] < -0.5 and current_position == -1:
        strategy['Signal'].iloc[i] = 1
        current_position = 0
    
    # Record current position
    strategy['Position'].iloc[i] = current_position

In [ ]:
# Plot buy and sell signals
buy_signals = strategy[strategy['Signal'] == 1]
sell_signals = strategy[strategy['Signal'] == -1]

plt.figure(figsize=(12, 6))
plt.plot(strategy.index, strategy['Z_Score'], label='Z-Score')
plt.axhline(y=0, color='black', linestyle='-')
plt.axhline(y=1.5, color='r', linestyle='--')
plt.axhline(y=-1.5, color='g', linestyle='--')
plt.axhline(y=0.5, color='orange', linestyle='--')
plt.axhline(y=-0.5, color='orange', linestyle='--')
plt.scatter(buy_signals.index, buy_signals['Z_Score'], marker='^', color='green', s=100, label='Buy Signal')
plt.scatter(sell_signals.index, sell_signals['Z_Score'], marker='v', color='red', s=100, label='Sell Signal')
plt.title('Trading Signals Based on Z-Score')
plt.xlabel('Date')
plt.ylabel('Z-Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/trading_signals.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Performance Analysis

Calculate returns and performance metrics for the strategy.

In [ ]:
# Calculate returns
strategy['BTC_Return'] = np.log(strategy['BTC'] / strategy['BTC'].shift(1))
strategy['ETH_Return'] = np.log(strategy['ETH'] / strategy['ETH'].shift(1))
strategy['Spread_Return'] = strategy['ETH_Return'] - strategy['Hedge_Ratio'].shift(1) * strategy['BTC_Return']
strategy['Strategy_Return'] = strategy['Spread_Return'] * strategy['Position'].shift(1)
strategy['Cum_Return'] = strategy['Strategy_Return'].fillna(0).cumsum().apply(np.exp)

In [ ]:
# Calculate trading statistics
total_trades = (strategy['Signal'] != 0).sum()
winning_trades = ((strategy['Signal'] == 1) & (strategy['Strategy_Return'].shift(-1) > 0)).sum() + \
                ((strategy['Signal'] == -1) & (strategy['Strategy_Return'].shift(-1) < 0)).sum()
if total_trades > 0:
    win_rate = winning_trades / total_trades
else:
    win_rate = 0

In [ ]:
# Calculate performance metrics
print("\nCalculating performance metrics...")
total_return = strategy['Cum_Return'].iloc[-1] - 1
annual_return = (1 + total_return) ** (365 / (len(strategy) / 24)) - 1  # Annualized hourly data
sharpe_ratio = strategy['Strategy_Return'].mean() / strategy['Strategy_Return'].std() * np.sqrt(365 * 24)  # Annualized
max_drawdown = (strategy['Cum_Return'] / strategy['Cum_Return'].cummax() - 1).min()

print(f"Total Return: {total_return:.4f} ({total_return*100:.2f}%)")
print(f"Annualized Return: {annual_return:.4f} ({annual_return*100:.2f}%)")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Maximum Drawdown: {max_drawdown:.4f} ({max_drawdown*100:.2f}%)")
print(f"Win Rate: {win_rate:.4f} ({win_rate*100:.2f}%)")
print(f"Total Trades: {total_trades}")

In [ ]:
# Plot cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(strategy.index, strategy['Cum_Return'])
plt.title('Pairs Trading Strategy Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, alpha=0.3)
plt.savefig('docs/cumulative_returns.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Comparison to Buy and Hold

Compare the pairs trading strategy to simple buy-and-hold approaches.

In [ ]:
# Define ticker names for easier reference
ticker_1 = 'BTC'
ticker_2 = 'ETH'

# Rename your test data to match the expected variable names
test_data = pd.DataFrame({
    ticker_1: btc_test,
    ticker_2: eth_test
})

In [ ]:
# Calculate buy-and-hold performance
btc_buy_hold = test_data[ticker_1] / test_data[ticker_1].iloc[0]
eth_buy_hold = test_data[ticker_2] / test_data[ticker_2].iloc[0]

# Create comparison dataframe
comparison = pd.DataFrame(index=strategy.index)
comparison['Pairs Strategy'] = strategy['Cum_Return']
comparison['BTC Buy & Hold'] = btc_buy_hold.loc[strategy.index] / btc_buy_hold.loc[strategy.index[0]]
comparison['ETH Buy & Hold'] = eth_buy_hold.loc[strategy.index] / eth_buy_hold.loc[strategy.index[0]]

In [ ]:
# Calculate buy-and-hold statistics
btc_return = comparison['BTC Buy & Hold'].iloc[-1] - 1
eth_return = comparison['ETH Buy & Hold'].iloc[-1] - 1
btc_sharpe = np.log(comparison['BTC Buy & Hold'] / comparison['BTC Buy & Hold'].shift(1)).mean() / \
            np.log(comparison['BTC Buy & Hold'] / comparison['BTC Buy & Hold'].shift(1)).std() * np.sqrt(252)
eth_sharpe = np.log(comparison['ETH Buy & Hold'] / comparison['ETH Buy & Hold'].shift(1)).mean() / \
            np.log(comparison['ETH Buy & Hold'] / comparison['ETH Buy & Hold'].shift(1)).std() * np.sqrt(252)

print("\nBuy & Hold Performance:")
print(f"BTC Return: {btc_return:.4f} ({btc_return*100:.2f}%)")
print(f"ETH Return: {eth_return:.4f} ({eth_return*100:.2f}%)")
print(f"BTC Sharpe: {btc_sharpe:.4f}")
print(f"ETH Sharpe: {eth_sharpe:.4f}")

In [ ]:
# Plot strategy vs buy and hold
plt.figure(figsize=(12, 6))
plt.plot(comparison.index, comparison['Pairs Strategy'], label='Pairs Strategy')
plt.plot(comparison.index, comparison['BTC Buy & Hold'], label='BTC Buy & Hold')
plt.plot(comparison.index, comparison['ETH Buy & Hold'], label='ETH Buy & Hold')
plt.title('Strategy vs Buy & Hold Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('strategy_vs_buyhold.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate drawdown
cum_returns = strategy['Cum_Return']
running_max = cum_returns.cummax()
drawdown = (cum_returns / running_max) - 1
max_drawdown = drawdown.min()
print(f"Maximum Drawdown: {max_drawdown:.4f} ({max_drawdown*100:.2f}%)")

In [ ]:
# Plot drawdowns
plt.figure(figsize=(12, 6))
plt.plot(strategy.index, drawdown * 100)
plt.fill_between(strategy.index, drawdown * 100, 0, color='red', alpha=0.3)
plt.title('Strategy Drawdowns (%)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Drawdown (%)')
plt.grid(True)
plt.savefig('drawdowns.png', dpi=300, bbox_inches='tight')
plt.show()

## **12. Market Regime Analysis**

Let's analyze how the correlation between BTC and ETH changes over time, which can impact the performance of our pairs trading strategy.

In [ ]:
# Calculate 90-day rolling correlation between BTC and ETH
rolling_corr = strategy[ticker_1].rolling(90).corr(strategy[ticker_2])

# Plot rolling correlation
plt.figure(figsize=(12, 6))
plt.plot(rolling_corr.index, rolling_corr)
plt.axhline(rolling_corr.mean(), color='red', linestyle='--', label=f'Mean: {rolling_corr.mean():.4f}')
plt.title('90-Day Rolling Correlation between BTC and ETH', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Correlation')
plt.legend()
plt.grid(True)
plt.savefig('rolling_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Define strategy_return
strategy_return = total_return

In [ ]:
# Create overall comparison statistics
comparison_stats = pd.DataFrame({
    'Metric': ['Total Return', 'Annualized Return', 'Sharpe Ratio', 'Maximum Drawdown'],
    'Pairs Strategy': [
        f"{strategy_return:.2%}", 
        f"{annual_return:.2%}", 
        f"{sharpe_ratio:.4f}", 
        f"{max_drawdown:.2%}"
    ],
    'BTC Buy & Hold': [
        f"{btc_return:.2%}", 
        f"{(1 + btc_return) ** (252 / len(strategy)) - 1:.2%}", 
        f"{btc_sharpe:.4f}", 
        f"{(comparison['BTC Buy & Hold'] / comparison['BTC Buy & Hold'].cummax() - 1).min():.2%}"
    ],
    'ETH Buy & Hold': [
        f"{eth_return:.2%}", 
        f"{(1 + eth_return) ** (252 / len(strategy)) - 1:.2%}", 
        f"{eth_sharpe:.4f}", 
        f"{(comparison['ETH Buy & Hold'] / comparison['ETH Buy & Hold'].cummax() - 1).min():.2%}"
    ]
})

print("\nStrategy Comparison Summary:")
print(comparison_stats)

## **13. Summary and Findings**

Let's summarize the key findings from our BTC-ETH pairs trading strategy analysis.

In [ ]:
print("Pairs trading strategy between BTC and ETH summary:")
print(f"1. The pair {'is' if pvalue < 0.05 else 'is not'} cointegrated during the training period")
print(f"2. The spread {'is' if adf_result[1] < 0.05 else 'is not'} stationary during the training period")
print(f"3. The strategy generated a {annual_return*100:.2f}% annualized return")
print(f"4. The Sharpe ratio is {sharpe_ratio:.4f}")
print(f"5. The strategy experienced a maximum drawdown of {max_drawdown*100:.2f}%")
print(f"6. The win rate is {win_rate*100:.2f}% ({winning_trades}/{total_trades} trades)")
print(f"7. The average correlation between BTC and ETH is {rolling_corr.mean():.4f}")

In [ ]:
# Save strategy data for further analysis
strategy.to_csv('btc_eth_pairs_trading_results.csv')
print("Strategy data saved to 'btc_eth_pairs_trading_results.csv'")